In [ ]:
import torch
from peft import (
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
)
from datasets import load_dataset

batch_size = 32
model_name_or_path = "roberta-large"
dataset_name_or_path = "stanfordnlp/snli"
task = "snli"
peft_type = PeftType.LORA
device = "cuda"

In [ ]:
datasets = load_dataset(dataset_name_or_path, split="train[:60%]")
datasets_test = load_dataset(dataset_name_or_path, split="test")

In [4]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    llm_int8_skip_modules=["classifier"]
)

model_name = "FacebookAI/roberta-large"
config = RobertaConfig.from_pretrained(
    model_name,
    num_labels=3,
)
tokenizer = RobertaTokenizer.from_pretrained(
    "FacebookAI/roberta-large",
    do_lower_case=False,
)
model = RobertaForSequenceClassification.from_pretrained(
    "FacebookAI/roberta-large",
    config=config,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
import torch.nn as nn
import transformers
from torch.utils.data import TensorDataset
from transformers.data.processors.utils import InputExample
from transformers.data.processors.glue import glue_convert_examples_to_features

LABEL_LIST = [0, 1, 2]

def _create_examples(dataset, set_type):
    """ Convert raw dataframe to a list of InputExample. Filter malformed examples
    """
    examples = []
    for index, item in enumerate(dataset):
        if item['label'] not in LABEL_LIST:
            continue
        if not isinstance(item['premise'], str) or not isinstance(item['hypothesis'], str):
            continue
        guid = f"{index}-{set_type}"
        examples.append(
            InputExample(guid=guid, text_a=item['premise'], text_b=item['hypothesis'], label=item['label']))
    return examples

def _dataset_to_features(dataset, set_type):
    """ Pre-process text. This method will:
    1) tokenize inputs
    2) cut or pad each sequence to MAX_SEQ_LENGHT
    3) convert tokens into ids

    The output will contain:
    `input_ids` - padded token ids sequence
    `attention mask` - mask indicating padded tokens
    `token_type_ids` - mask indicating the split between premise and hypothesis
    `label` - label
    """
    examples = _create_examples(dataset, set_type)

    #backward compatibility with older transformers versions
    legacy_kwards = {}
    from packaging import version
    if version.parse(transformers.__version__) < version.parse("2.9.0"):
        legacy_kwards = {
            "pad_on_left": False,
            "pad_token": tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
            "pad_token_segment_id": 0,
        }

    return glue_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        label_list=LABEL_LIST,
        max_length=128,
        output_mode="classification",
        **legacy_kwards,
    )

def _features_to_dataset(features):
    """ Convert features from `_df_to_features` into a single dataset
    """
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor(
        [f.attention_mask for f in features], dtype=torch.long
    )
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
    dataset = TensorDataset(
        all_input_ids, all_attention_mask, all_labels
    )

    return dataset


train_features = _dataset_to_features(datasets, "train")
test_features = _dataset_to_features(datasets_test, "test")

train_dataset = _features_to_dataset(train_features)
test_dataset = _features_to_dataset(test_features)

/home/david-stan/workspace/venv/lib/python3.10/site-packages/transformers/data/processors/glue.py:66: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


In [7]:
BATCH_SIZE = 256
MAX_PHYSICAL_BATCH_SIZE = 64

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from opacus.utils.uniform_sampler import UniformWithReplacementSampler

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, sampler=RandomSampler(test_dataset), batch_size=BATCH_SIZE)

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters count: {total_params:,}")


lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # our particular task is sequence classification
    inference_mode=False,  # Enable training mode
    r=8,  # Low-rank dimension
    lora_alpha=8,  # Alpha scaling factor
    lora_dropout=0.05,  # Dropout for LoRA layers
    target_modules=["query", "value"],
)

model_with_lora = get_peft_model(model, lora_config)
trainable_params = sum(p.numel() for p in model_with_lora.parameters() if p.requires_grad)
print(f"Total trainable parameters with LoRA: {trainable_params:,}")

# FFA-LoRA modification: freeze all adapter A matrices so that only B matrices are trainable
for name, param in model_with_lora.named_parameters():
    if "lora_A" in name or "classifier" in name:
        param.requires_grad = False

# LoRA
# for name, param in model_with_lora.named_parameters():
#     if "classifier" in name:
#         param.requires_grad = False

trainable_params = sum(p.numel() for p in model_with_lora.parameters() if p.requires_grad)
print(f"Total trainable parameters with LoRA after freezing matrix A: {trainable_params:,}")

Total parameters count: 53,151,747
Total trainable parameters with LoRA: 1,839,107
Total trainable parameters with LoRA after freezing matrix A: 393,216


In [ ]:
EPOCHS = 1
LOGGING_INTERVAL = 200 # once every how many steps we run evaluation cycle and report metrics
EPSILON = 6.0
DELTA = 1e-5 # probability of not achieving privacy guarantees
MAX_GRAD_NORM = 10.0

In [ ]:
def accuracy(preds, labels):
    return (preds == labels).float().mean().item()

def evaluate(model):
    model.eval()

    loss_arr = []
    accuracy_arr = []

    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[2]}

            outputs = model(**inputs)
            loss, logits = outputs[:2]

            preds = torch.argmax(logits, dim=1)
            labels = inputs['labels']

            loss_arr.append(loss.item())
            accuracy_arr.append(accuracy(preds, labels))

    model.train()
    avg_loss = sum(loss_arr) / len(loss_arr)
    avg_accuracy = sum(accuracy_arr) / len(accuracy_arr)
    return avg_loss, avg_accuracy

In [ ]:
import copy
import numpy as np
import torch
from torch.utils.data import DataLoader, Subset, RandomSampler
from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager
from tqdm import tqdm

# --- PARTITIONING ---

def partition_dataset_homogeneous(dataset: TensorDataset, num_clients=3):
    indices = np.random.permutation(len(dataset))
    splits = np.array_split(indices, num_clients)
    return [Subset(dataset, split) for split in splits]

def partition_dataset_heterogeneous(dataset: TensorDataset, num_clients=3):
    """
    3-class classification task, partitioning scheme:
      - Client 0: 90% of samples have label 0, 5% label 1, 5% label 2
      - Client 1: 90% label 1, 5% label 0, 5% label 2
      - Client 2: 90% label 2, 5% label 0, 5% label 1
    """
    # Extract labels as numpy array
    labels = np.array([sample.item() for sample in dataset.tensors[2]])
    indices = np.arange(len(labels))
    partitions = []
    
    proportions = [
        {0: 0.9, 1: 0.05, 2: 0.05},
        {1: 0.9, 0: 0.05, 2: 0.05},
        {2: 0.9, 0: 0.05, 1: 0.05},
    ]
    for client in range(num_clients):
        client_indices = []
        for label in [0, 1, 2]:
            label_indices = indices[labels == label]
            # Determine number of samples to pick for this label from client partition
            num_samples = int(len(label_indices) * proportions[client][label])
            if num_samples > 0:
                selected = np.random.choice(label_indices, size=num_samples, replace=False)
                client_indices.extend(selected)
        partitions.append(Subset(dataset, client_indices))
    return partitions


# client_train_partitions = partition_dataset_homogeneous(train_dataset, num_clients=3)

client_train_partitions = partition_dataset_heterogeneous(train_dataset, num_clients=3)

# consideration
client_test_partitions = [test_dataset] * 3

# --- FEDERATED TRAINING LOOP ---

def FedAvg(w):
    w_avg = copy.deepcopy(w[0])
    for k in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[k] += w[i][k]
        w_avg[k] = torch.div(w_avg[k], len(w))
    return w_avg


global_model = copy.deepcopy(model_with_lora).to(device)
global_model.train()

num_clients = 3

NUM_GLOBAL_ROUNDS = 20


for round in range(1, NUM_GLOBAL_ROUNDS + 1):
    print(f"Global Round {round}")
    client_params_list = []
    client_sample_counts = []

    for client_id in range(num_clients):
        print(f"  Client {client_id} training...")

        local_model = copy.deepcopy(global_model).to(device)
        local_model.train()
        # local dataloader
        local_train_loader = DataLoader(
            client_train_partitions[client_id],
            sampler=RandomSampler(client_train_partitions[client_id]),
            batch_size=BATCH_SIZE,
        )

        
        local_optimizer = torch.optim.SGD(local_model.parameters(), lr=0.2)
        
        privacy_engine = PrivacyEngine()

        local_model, local_optimizer, local_train_loader = privacy_engine.make_private_with_epsilon(
            module=local_model,
            optimizer=local_optimizer,
            data_loader=local_train_loader,
            target_delta=DELTA,
            target_epsilon=EPSILON,
            epochs=EPOCHS,  # local training epochs
            max_grad_norm=MAX_GRAD_NORM,
        )

        for epoch in range(1, EPOCHS+1):
            local_losses = []
            with BatchMemoryManager(
                data_loader=local_train_loader,
                max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE,
                optimizer=local_optimizer
            ) as memory_safe_loader:
                for step, batch in enumerate(tqdm(memory_safe_loader, desc=f"Client {client_id} training")):
                    local_optimizer.zero_grad()
                    batch = tuple(t.to(device) for t in batch)
                    inputs = {
                        'input_ids': batch[0],
                        'attention_mask': batch[1],
                        'labels': batch[2],
                    }
                    outputs = local_model(**inputs)
                    loss = outputs[0]
                    loss.backward()
                    local_losses.append(loss.item())
                    local_optimizer.step()
                    if step > 0 and step % LOGGING_INTERVAL == 0:
                        train_loss = np.mean(local_losses)
                        eps = privacy_engine.get_epsilon(DELTA)

                        eval_loss, eval_accuracy = evaluate(local_model)

                        print(
                            f"Step: {step} | "
                            f"Train loss: {train_loss:.3f} | "
                            f"Eval loss: {eval_loss:.3f} | "
                            f"Eval accuracy: {eval_accuracy:.3f} | "
                            f"ɛ: {eps:.2f} "
                        )

        print(f"    Local training loss: {np.mean(local_losses):.3f}")
        # get parameters
        local_params = get_peft_model_state_dict(local_model.base_model)
        client_params_list.append(local_params)
        client_sample_counts.append(len(client_train_partitions[client_id]))

    # aggregate
    aggregated_params = FedAvg(client_params_list)
    # update ~.base_model!!!
    set_peft_model_state_dict(global_model.base_model, aggregated_params)

    # eval global
    global_eval_loss, global_eval_accuracy = evaluate(global_model)

    print(
        f"After Global Round {round}: Eval loss: {global_eval_loss:.3f} | "
        f"Eval accuracy: {global_eval_accuracy:.3f} | ɛ: {eps:.2f}"
    )

Global Round 1
  Client 0 training...


/home/david-stan/workspace/venv/lib/python3.10/site-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/home/david-stan/workspace/venv/lib/python3.10/site-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
Client 0 training:   0%|          | 0/2865 [00:00<?, ?it/s]/home/david-stan/workspace/venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


Step: 200 | Train loss: 0.639 | Eval loss: 2.126 | Eval accuracy: 0.342 | ɛ: 3.93 


Client 0 training:  14%|█▍        | 401/2865 [02:11<2:58:53,  4.36s/it]

Step: 400 | Train loss: 0.536 | Eval loss: 2.183 | Eval accuracy: 0.342 | ɛ: 4.38 


...

Step: 3000 | Train loss: 0.255 | Eval loss: 0.868 | Eval accuracy: 0.686 | ɛ: 5.92 


Client 2 training: 3202it [09:17,  1.55s/it]

Step: 3200 | Train loss: 0.253 | Eval loss: 0.903 | Eval accuracy: 0.674 | ɛ: 5.98 


Client 2 training: 3234it [09:22,  5.75it/s]


    Local training loss: 0.254
After Global Round 5: Eval loss: 0.512 | Eval accuracy: 0.809 | ɛ: 5.98


In [15]:
evaluate(global_model)

(0.5127203525641025, 0.8088274582838401)